In [1]:
from sqlalchemy import create_engine, text

In [5]:
from sqlalchemy import create_engine, text
from bs4 import BeautifulSoup
db_url = "mysql+pymysql://root:@localhost:3306/sevensix_website_dump"
engine = create_engine(db_url, echo=True)
# --- Modified Preprocessing Function ---
def clean_wordpress_content_and_get_media(html_content):
    """Strips HTML tags and extracts image and YouTube URLs from a string."""
    clean_text = ""
    image_urls = []
    youtube_urls = []
    
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find all <img> tags and extract their 'src' attribute
        for img in soup.find_all('img'):
            if 'src' in img.attrs:
                image_urls.append(img['src'])
                
        # Find all <iframe> tags and extract YouTube links
        for iframe in soup.find_all('iframe'):
            if 'src' in iframe.attrs and 'youtube.com' in iframe['src']:
                youtube_urls.append(iframe['src'])
                
        for div in soup.find_all('div', class_='wp-block-embed__wrapper'):
            youtube_url = div.get_text(strip=True)
            if 'youtube.com' in youtube_url:
                youtube_urls.append(youtube_url)
        
        # 3. <a>タグ内のリンク
        for a_tag in soup.find_all('a', href=True):
            if 'youtube.com' in a_tag['href']:
                youtube_urls.append(a_tag['href'])
                
        # Get the text content after finding the media links
        clean_text = soup.get_text(separator=' ', strip=True)

    return clean_text, image_urls, youtube_urls
# -------------------------------------

# Database connection and query setup (same as before)

sql_query = text("""
    SELECT
        p.post_content as post_content,
        p.post_title as post_title,
        pm.meta_value as post_summary,
                 p.post_name as post_url
    FROM
        wp_posts AS p
    JOIN
        wp_postmeta AS pm ON p.ID = pm.post_id
    WHERE
        p.post_type = 'products'
        AND pm.meta_key = 'cfp_fname_1'
    LIMIT 5;
""")

with engine.connect() as connection:
    result = connection.execute(sql_query)
    
    print("Headings:", result.keys())
    print("Rows:")
    for row in result:
        # Call the new function that returns three values
        cleaned_content, image_urls, youtube_urls = clean_wordpress_content_and_get_media(row.post_content)
        
        print(f"Post Title: {row.post_title}")
        print(f"Cleaned Content: {cleaned_content}")
        print(f"Post Summary: {row.post_summary}")
        print(f"Post URL: {row.post_url}")
        
        # Check if any images were found and print them
        if image_urls:
            print("Image URLs found:")
            for url in image_urls:
                print(f"  - {url}")
        
        # Check if any YouTube videos were found and print them
        if youtube_urls:
            print("YouTube URLs found:")
            for url in youtube_urls:
                print(f"  - {url}")
        
        print("-" * 20)

2025-08-11 16:19:12,213 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-08-11 16:19:12,214 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-11 16:19:12,217 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-08-11 16:19:12,218 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-11 16:19:12,220 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-08-11 16:19:12,220 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-11 16:19:12,222 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-11 16:19:12,222 INFO sqlalchemy.engine.Engine 
    SELECT
        p.post_content as post_content,
        p.post_title as post_title,
        pm.meta_value as post_summary,
                 p.post_name as post_url
    FROM
        wp_posts AS p
    JOIN
        wp_postmeta AS pm ON p.ID = pm.post_id
    WHERE
        p.post_type = 'products'
        AND pm.meta_key = 'cfp_fname_1'
    LIMIT 5;

2025-08-11 16:19:12,222 INFO sqlalchemy.engine.Engine [generated in 0.00074s] {}
Headings: R

In [4]:
from src.website_data_transformation.wordpress_data_processor import WordpressDataProcessor
from dotenv import load_dotenv
load_dotenv()
processor = WordpressDataProcessor()
processor.process()

TypeError: WordpressDataProcessor.process() missing 1 required positional argument: 'l'